In [ ]:
from pyspark.sql.functions import *

In [ ]:
import ibmos2spark, os
# @hidden_cell

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_1 = '#################################'
else:
    endpoint_1 = '#################################''

credentials = {
    'endpoint': endpoint_1,
    'service_id': '#################################'',
    'iam_service_endpoint': '#################################'',
    'api_key': '#################################''
}

configuration_name = '#################################''
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()


In [ ]:
compra = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('compra.csv', ',"############################'))

compraentrega = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('compraentrega.csv', ',"############################'))

compraentregasku = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('compraentregasku.csv', ',"############################'))

compraformapagamento = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('compraformapagamento.csv', ',"############################'))

formapagamento = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('formapagamento.csv', ',"############################'))

compraentregastatus = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('compraentregastatus.csv', ',"############################'))

lojista = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('lojista.csv', ',"############################'))

skulojista = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('skulojista.csv', ',"############################'))

skucategoria = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('skucategoria.csv', ',"############################'))

visaolojistavendanps = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('visaolojistavendanps.csv', ',"############################'))

In [ ]:
def rel_time(df, date_col, end_date=False):
    if end_date == False:
        df = df.withColumn("filter_12m", (from_utc_timestamp(current_timestamp(), "Brazil/East") - expr("INTERVAL 12 MONTHS"))) \
                .withColumn("filter_9m", end_date - expr("INTERVAL 9 MONTHS")) \
                .withColumn("filter_6m", end_date - expr("INTERVAL 6 MONTHS")) \
                .withColumn("filter_3m", end_date - expr("INTERVAL 3 MONTHS")) \
                .filter(col(date_col) >= col("filter_12m"))
    else:
        df = df.withColumn("filter_12m", end_date - expr("INTERVAL 12 MONTHS")) \
                .withColumn("filter_9m", end_date - expr("INTERVAL 9 MONTHS")) \
                .withColumn("filter_6m", end_date - expr("INTERVAL 6 MONTHS")) \
                .withColumn("filter_3m", end_date - expr("INTERVAL 3 MONTHS")) \
                .filter(col(date_col) >= col("filter_12m"))
    return df

In [ ]:
compra_filtrado = rel_time(compra, date_col='datacriacaoregistro', end_date='2021-01-01')

In [ ]:
compra12m = compra_filtrado.where(col('datacriacaoregistro').between(col('filter_12m'), col('filter_9m')))
compra9m = compra_filtrado.where(col('datacriacaoregistro').between(col('filter_9m'), col('filter_6m')))
compra6m = compra_filtrado.where(col('datacriacaoregistro').between(col('filter_6m'), col('filter_3m')))
compra3m = compra_filtrado.where(col('datacriacaoregistro')>=col('filter_3m'))

In [ ]:
df_formapagamento = formapagamento.select('idformapagamento', col('nome').alias('DescricaoFormaPagamento'), 'idbandeira')\
                        .distinct()\
                        .cache()

df_compraformapagamento = compraformapagamento.select('idformapagamento','idcompra','quantidadeparcelas','idbandeira')\
                                              .join(df_formapagamento, ['idformapagamento', 'idbandeira'])\
                                              .distinct()\
                                              .cache()

df_compraentregasku = compraentregasku.select('idcompraentrega','idcompraentregasku','valorvendaunidademenoscupomnominal','valorfretecomdesconto', 'idbandeira')\
                        .distinct()\
                        .groupBy('idcompraentrega', 'idbandeira')\
                        .agg(sum(col('valorvendaunidademenoscupomnominal') + col('valorfretecomdesconto')).alias('sum_preço'),
                            count('idcompraentregasku').alias('skus_vendidos'))\
                        .cache()

df_compraentrega = compraentrega.select('idcompra', 'idlojista', 'idcompraentrega', 'idbandeira', 'idcompraentregastatus')\
                        .distinct()\
                        .join(df_compraentregasku, ['idcompraentrega','idbandeira'])\
                        .cache()

df_lojista = lojista.select('idlojista','idbandeira', col('classificacao').alias('classificacao_lojista'), 'porcentagempositivo','quantidadereviews')\
                        .distinct()\
                        .cache()

df_skucategoria = skucategoria.select('idsku', 'nomedepartamento')

df_agg_skuloj = skulojista.join(df_skucategoria, ['idsku'])\
                          .join(visaolojistavendanps, ['idlojista', 'idbandeira'])\
                            .groupBy('idlojista', 'idbandeira')\
                          .agg(avg(col('classificacao')).alias('avg_classificacao_skus_vendidos'),
                               max(col('precovenda')).alias('max_precosku'),
                              avg(col('TxCalculoNPS')).alias('avg_TxCalculoNPS'),
                               sum(col('QtResposta')).alias('sum_QtResposta'),
                              max(col('nomedepartamento')).alias('principal_categoria'))\
                          .join(df_lojista, ['idlojista', 'idbandeira'])\

In [ ]:
df_compra12 = compra12m.select('idcompra','valortotalcomdesconto','idbandeira','datacriacaoregistro','filter_12m', 'filter_9m', 'filter_6m', 'filter_3m')\
                        .distinct().cache()

df_compra9 = compra9m.select('idcompra','valortotalcomdesconto','idbandeira','datacriacaoregistro','filter_12m', 'filter_9m', 'filter_6m', 'filter_3m')\
                        .distinct().cache()

df_compra6 = compra6m.select('idcompra','valortotalcomdesconto','idbandeira','datacriacaoregistro','filter_12m', 'filter_9m', 'filter_6m', 'filter_3m')\
                        .distinct().cache()

df_compra3 = compra3m.select('idcompra','valortotalcomdesconto','idbandeira','datacriacaoregistro','filter_12m', 'filter_9m', 'filter_6m', 'filter_3m')\
                        .distinct().cache()

In [ ]:
dfjoin12 =  df_compraentrega.join(df_compra12,['idcompra','idbandeira'])
dfjoin9 =  df_compraentrega.join(df_compra9,['idcompra','idbandeira'])
dfjoin6 =  df_compraentrega.join(df_compra6,['idcompra','idbandeira'])
dfjoin3 =  df_compraentrega.join(df_compra3,['idcompra','idbandeira'])

In [ ]:
dfgrp12 = dfjoin12.withColumn('FlagEntregue', when(col('idcompraentregastatus')=='ENT', lit(1)).otherwise(''))\
                                .groupBy('idlojista', 'idbandeira')\
                  .agg(countDistinct(col('idcompra')).alias('qtdetotal_pedidos_12m'),
                       sum(col('skus_vendidos')).alias('QtTotalSkus_12m'),
                       sum(col('sum_preço')).alias('VrTotalVendido_12m'),
                      sum(col('FlagEntregue')).alias('QtTotalCompraEntregaENT_12m'))

dfgrp9 = dfjoin9.withColumn('FlagEntregue', when(col('idcompraentregastatus')=='ENT', lit(1)).otherwise(''))\
                                .groupBy('idlojista', 'idbandeira')\
                  .agg(countDistinct(col('idcompra')).alias('qtdetotal_pedidos_9m'),
                       sum(col('skus_vendidos')).alias('QtTotalSkus_9m'),
                       sum(col('sum_preço')).alias('VrTotalVendido_9m'),
                      sum(col('FlagEntregue')).alias('QtTotalCompraEntregaENT_9m'))

dfgrp6 = dfjoin6.withColumn('FlagEntregue', when(col('idcompraentregastatus')=='ENT', lit(1)).otherwise(''))\
                                .groupBy('idlojista', 'idbandeira')\
                  .agg(countDistinct(col('idcompra')).alias('qtdetotal_pedidos_6m'),
                       sum(col('skus_vendidos')).alias('QtTotalSkus_6m'),
                       sum(col('sum_preço')).alias('VrTotalVendido_6m'),
                      sum(col('FlagEntregue')).alias('QtTotalCompraEntregaENT_6m'))

dfgrp3 = dfjoin3.withColumn('FlagEntregue', when(col('idcompraentregastatus')=='ENT', lit(1)).otherwise(''))\
                                .groupBy('idlojista', 'idbandeira')\
                  .agg(countDistinct(col('idcompra')).alias('qtdetotal_pedidos_3m'),
                       sum(col('skus_vendidos')).alias('QtTotalSkus_3m'),
                       sum(col('sum_preço')).alias('VrTotalVendido_3m'),
                      sum(col('FlagEntregue')).alias('QtTotalCompraEntregaENT_3m'))

In [ ]:
df_final = df_agg_skuloj.join(dfgrp12, ['idlojista','idbandeira'], how='left')\
                        .join(dfgrp9, ['idlojista','idbandeira'], how='left')\
                        .join(dfgrp6, ['idlojista','idbandeira'], how='left')\
                        .join(dfgrp3, ['idlojista','idbandeira'], how='left')\
                        .withColumn('id', concat(col('idlojista'), lit('_'), concat('idbandeira')))

pd_final_df = df_final.toPandas()

In [ ]:
df_final.count()

In [ ]:
from project_lib import Project
project = Project(sc,"############################","################################")
project.save_data(file_name = "agg_base.json", data = pd_final_df.to_json(orient='records'))

In [ ]:
from project_lib import Project
project = Project(sc,"#############################","###############################")
project.save_data(file_name = "agg_base.csv", data = pd_final_df.to_csv(index=False))